In [35]:
import warnings
import pandas as pd
from tqdm import tqdm
warnings.filterwarnings('ignore')

In [140]:
def load_skills(path: str, size: int = 20, seperator: str = 'skill') -> pd.DataFrame:

    df_ = pd.read_csv(path)
    print(f'Frequency of top 20 skills before preprocess: {df_["skill"].value_counts()[:20].values.sum()}')
    df_['skill'] = df_['skill'].apply(lambda x: x.strip())

    df_.loc[df_['skill'] == 'Web Geliştirme', 'skill'] = 'Web Development'
    df_.loc[df_['skill'] == 'Web Uygulamaları', 'skill'] = 'Web Applications'
    df_.loc[df_['skill'] == 'Proje Yönetimi', 'skill'] = 'Project Management'
    df_.loc[df_['skill'] == 'Programlama', 'skill'] = 'Programming'
    df_.loc[df_['skill'] == 'Object-Oriented Programming (OOP)', 'skill'] = 'OOP'
    df_.loc[df_['skill'] == 'Object Oriented Programming (OOP)', 'skill'] = 'OOP'
    df_.loc[df_['skill'] == 'OOP (Nesne Yönelimli Programlama)', 'skill'] = 'OOP'
    df_.loc[df_['skill'] == 'OOP(Object Oriented Programming)', 'skill'] = 'OOP'
    df_.loc[df_['skill'] == 'OOP (Object Oriented Programming)', 'skill'] = 'OOP'
    df_.loc[df_['skill'] == 'Objektorientierte Programmierung (OOP)', 'skill'] = 'OOP'
    df_.loc[df_['skill'] == 'Object-Oriented Programming(OOP)', 'skill'] = 'OOP'
    df_.loc[df_['skill'] == 'OOP(Object Orianted Programming)', 'skill'] = 'OOP'
    df_.loc[df_['skill'] == 'OOP (Nesne Yönelimli Programlama', 'skill'] = 'OOP'
    df_.loc[df_['skill'] == 'OOP programming and implementing design patterns', 'skill'] = 'OOP'
    df_.loc[df_['skill'] == 'OOP(Object-Oriented Programming)', 'skill'] = 'OOP'
    df_.loc[df_['skill'] == 'Nesne Yönelimli Programlama(OOP)', 'skill'] = 'OOP'
    df_.loc[df_['skill'] == 'Object Oriented Programming(OOP)', 'skill'] = 'OOP'
    df_.loc[df_['skill'] == 'Programación orientada a objetos (OOP)', 'skill'] = 'OOP'
    df_.loc[df_['skill'] == 'OOP ( Object Oriented Programming )', 'skill'] = 'OOP'
    df_.loc[df_['skill'] == 'OOP (Object-Oriented Programming)', 'skill'] = 'OOP'
    df_.loc[df_['skill'] == 'OOP ( Object - Oriented Programming )', 'skill'] = 'OOP'
    df_.loc[df_['skill'] == 'OOP(Nesne Yönelimli Programlama)', 'skill'] = 'OOP'
    df_.loc[df_['skill'] == 'Object-oriented Programming (OOP)', 'skill'] = 'OOP'
    df_.loc[df_['skill'] == 'C, C++ (OOP), Linux', 'skill'] = 'OOP'
    df_.loc[df_['skill'] == 'OOP Design', 'skill'] = 'OOP'
    df_.loc[df_['skill'] == 'Advanced OOP', 'skill'] = 'OOP'
    df_.loc[df_['skill'] == 'Python (Programming Language)', 'skill'] = 'Python'
    df_.loc[df_['skill'] == 'Python (Programmiersprache)', 'skill'] = 'Python'
    df_.loc[df_['skill'] == 'Python Programming Language', 'skill'] = 'Python'
    df_.loc[df_['skill'] == 'Ptyhon', 'skill'] = 'Python'
    df_.loc[df_['skill'] == 'Phyton', 'skill'] = 'Python'
    df_.loc[df_['skill'] == 'C (Programming Language)', 'skill'] = 'C'
    df_.loc[df_['skill'] == 'Swift (Programming Language)', 'skill'] = 'Swift'
    df_.loc[df_['skill'] == 'Go (Programming Language)', 'skill'] = 'Go'
    df_.loc[df_['skill'] == 'R (Programming Language)', 'skill'] = 'R'
    df_.loc[df_['skill'] == 'data science', 'skill'] = 'Data Science'
    df_.loc[df_['skill'] == 'Data science', 'skill'] = 'Data Science'
    df_.loc[df_['skill'] == 'Microsoft Teknolojileri', 'skill'] = 'Microsoft Technologies'
    df_.loc[df_['skill'] == 'Microsoft Sunucular', 'skill'] = 'Microsoft Servers'
    df_.loc[df_['skill'] == 'microsoft', 'skill'] = 'Microsoft'
    df_.loc[df_['skill'] == 'MsSQL database', 'skill'] = 'Microsoft SQL Server'
    df_.loc[df_['skill'] == 'MsSQL Server', 'skill'] = 'Microsoft SQL Server'
    df_.loc[df_['skill'] == 'MsSQL', 'skill'] = 'Microsoft SQL Server'
    df_.loc[df_['skill'] == 'Amazon Web Services (AWS)', 'skill'] = 'AWS'
    df_.loc[df_['skill'] == 'Amazon Web Hizmetleri (AWS)', 'skill'] = 'AWS'
    df_.loc[df_['skill'] == 'Amazon Web Services', 'skill'] = 'AWS'
    df_.loc[df_['skill'] == 'machine learning', 'skill'] = 'Machine Learning'
    df_.loc[df_['skill'] == 'Machine learning', 'skill'] = 'Machine Learning'
    df_.loc[df_['skill'] == 'Makine Öğrenmesi/Machine Learning', 'skill'] = 'Machine Learning'
    df_.loc[df_['skill'] == 'Artificial Intelligence (AI)', 'skill'] = 'Artificial Intelligence'
    df_.loc[df_['skill'] == 'Artificial intelligence', 'skill'] = 'Artificial Intelligence'
    df_.loc[df_['skill'] == 'Yazılım Proje Yönetimi', 'skill'] = 'Software Project Management'
    df_.loc[df_['skill'] == 'Proje Planlama', 'skill'] = 'Project Planning'
    df_.loc[df_['skill'] == 'İngilizce', 'skill'] = 'English'
    df_.loc[df_['skill'] == 'english', 'skill'] = 'English'
    df_.loc[df_['skill'] == 'git', 'skill'] = 'Git'
    df_.loc[df_['skill'] == 'github', 'skill'] = 'GitHub'
    df_.loc[df_['skill'] == 'Yazılım Geliştirme', 'skill'] = 'Software Development'
    df_.loc[df_['skill'] == 'Yazılım Mühendisliği', 'skill'] = 'Software Engineering'
    df_.loc[df_['skill'] == 'Yazılım Mühendisleri', 'skill'] = 'Software Engineering'
    df_.loc[df_['skill'] == 'Yazılım Tasarımı', 'skill'] = 'Software Design'
    df_.loc[df_['skill'] == 'Yazılım', 'skill'] = 'Software'
    df_.loc[df_['skill'] == 'yazılım', 'skill'] = 'Software'
    df_.loc[df_['skill'] == 'Açık Kaynak Yazılımı', 'skill'] = 'Open Source Software'
    df_.loc[df_['skill'] == 'Yazılım Geliştirme Metodolojileri', 'skill'] = 'Software Development Methodologies'
    df_.loc[df_['skill'] == 'Yazılım Kalitesi', 'skill'] = 'Software Quality'
    df_.loc[df_['skill'] == 'Yazılım Konfigürasyon Yönetimi', 'skill'] = 'Software Configuration Management'
    df_.loc[df_['skill'] == 'Yazılım Çözümleri', 'skill'] = 'Software Solutions'
    df_.loc[df_['skill'] == 'Yazılım Dokümantasyonu', 'skill'] = 'Software Documentation'
    df_.loc[df_['skill'] == 'Yazılım Kalite Güvencesi', 'skill'] = 'Software Quality Assurance'
    df_.loc[df_['skill'] == 'matlab', 'skill'] = 'Matlab'
    df_.loc[df_['skill'] == 'Php', 'skill'] = 'PHP'   
    df_.loc[df_['skill'] == 'java', 'skill'] = 'Java' 
    df_.loc[df_['skill'] == 'java dili', 'skill'] = 'Java' 
    df_.loc[df_['skill'] == 'javascript', 'skill'] = 'JavaScript'
    df_.loc[df_['skill'] == 'java script', 'skill'] = 'JavaScript'
    df_.loc[df_['skill'] == 'html', 'skill'] = 'HTML'
    df_.loc[df_['skill'] == 'My Sql', 'skill'] = 'MySQL'
    df_.loc[df_['skill'] == 'English B2', 'skill'] = 'English'
    df_.loc[df_['skill'] == 'English C1', 'skill'] = 'English'
    df_.loc[df_['skill'] == 'jquery', 'skill'] = 'jQuery' 
    df_.loc[df_['skill'] == 'Mssql', 'skill'] = 'Microsoft SQL Server'
    df_.loc[df_['skill'] == 'mssql', 'skill'] = 'Microsoft SQL Server'
    df_.loc[df_['skill'] == 'proje yönetimi', 'skill'] = 'Project Management'
    df_.loc[df_['skill'] == 'project management', 'skill'] = 'Project Management'
    
    print(f'Frequency of top 20 skills after preprocess: {df_["skill"].value_counts()[:20].values.sum()}')

    df_ = df_.drop_duplicates()
    
    most_freq_skills = df_['skill'].value_counts().keys()[:size].tolist()
    grouped = df_.groupby(by='user_id', as_index=False).agg({'skill': 'unique'})
    for skill in tqdm(most_freq_skills):
        grouped[f'{seperator}_{skill}'] = grouped['skill'].apply(lambda x: 1 if skill in x else 0)

    grouped = grouped.drop(columns=['skill'], axis=1)
    return grouped


In [150]:
#df = load_skills('../../../datasets/garanti-bbva-data-camp/skills.csv')

#df

In [151]:
#df['skill'].value_counts()[20:40]

In [152]:
#df.loc[df['skill'].str.contains('web'), 'skill'].value_counts()[:20]